In [ ]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

**----------Load The Data---------**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [ ]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 6 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [ ]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/hepmass_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [ ]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, SQ)
]

In [ ]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**--------QuickSelect Algorithm----------------**

In [ ]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

**----------VP Tree---------------------**

In [ ]:
import numpy as np
from collections import deque
#*-----------------------------------------------------------------------------

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None
#*-----------------------------------------------------------------------------

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon

    #*-----------------------------------------------------------------------------
    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    #*-----------------------------------------------------------------------------
    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

    #*-----------------------------------------------------------------------------
    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    #*-----------------------------------------------------------------------------
    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        # Dynamically adjust the node size for homogeneous point storage
        if n == 2*self.leaf_size:
            self.leaf_size = n

        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
#*------------------------------------------------------------------------------------------

--------------

**------VP Tree Node Size Optimization For Rebuild------**

**------------Run 1---------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 52.93it/s]


-------------------------------------------------------------
Build Time: 17.5225 sec, Search Time: 0.6326 sec
Leaf Size: 100, Total Time: 18.1551 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 64.49it/s]


-------------------------------------------------------------
Build Time: 16.4674 sec, Search Time: 0.5076 sec
Leaf Size: 200, Total Time: 16.9750 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.2181 sec, Search Time: 0.4293 sec
Leaf Size: 9,800, Total Time: 11.6474 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 73.16it/s]


-------------------------------------------------------------
Build Time: 11.3326 sec, Search Time: 0.4315 sec
Leaf Size: 9,900, Total Time: 11.7641 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 72.48it/s]

-------------------------------------------------------------
Build Time: 11.4542 sec, Search Time: 0.4350 sec
Leaf Size: 10,000, Total Time: 11.8892 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 17.5225 sec, Search Time: 0.6326 sec, Total Time: 18.1551 sec
Leaf Size: 200, Build Time: 16.4674 sec, Search Time: 0.5076 sec, Total Time: 16.9750 sec
Leaf Size: 300, Build Time: 15.2695 sec, Search Time: 0.3994 sec, Total Time: 15.6690 sec
Leaf Size: 400, Build Time: 15.2418 sec, Search Time: 0.4052 sec, Total Time: 15.6471 sec
Leaf Size: 500, Build Time: 14.4775 sec, Search Time: 0.3658 sec, Total Time: 14.8432 sec
Leaf Size: 600, Build Time: 14.2452 sec, Search Time: 0.3629 sec, Total Time: 14.6080 sec
Leaf Size: 700, Build Time: 14.4268 sec, Search Time: 0.3572 sec, Total Time: 14.7840 sec
Leaf Size: 800, Build Time: 14.4874 sec, Search Time: 0.3537 sec, Total Time: 14.8412 sec
Leaf Size: 900, Build Time: 14.4458 sec, Search Time: 0.3621 sec, Total Time: 14.80

**---------------------Run 2---------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 39.71it/s]


-------------------------------------------------------------
Build Time: 18.1642 sec, Search Time: 0.7188 sec
Leaf Size: 100, Total Time: 18.8830 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 54.58it/s]


-------------------------------------------------------------
Build Time: 17.2585 sec, Search Time: 0.5457 sec
Leaf Size: 200, Total Time: 17.8042 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 11.4454 sec, Search Time: 0.4546 sec
Leaf Size: 9,800, Total Time: 11.9000 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 67.09it/s]


-------------------------------------------------------------
Build Time: 11.3915 sec, Search Time: 0.4571 sec
Leaf Size: 9,900, Total Time: 11.8486 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 67.44it/s]

-------------------------------------------------------------
Build Time: 11.6959 sec, Search Time: 0.4580 sec
Leaf Size: 10,000, Total Time: 12.1539 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 18.1642 sec, Search Time: 0.7188 sec, Total Time: 18.8830 sec
Leaf Size: 200, Build Time: 17.2585 sec, Search Time: 0.5457 sec, Total Time: 17.8042 sec
Leaf Size: 300, Build Time: 16.5410 sec, Search Time: 0.4792 sec, Total Time: 17.0202 sec
Leaf Size: 400, Build Time: 16.7885 sec, Search Time: 0.4604 sec, Total Time: 17.2489 sec
Leaf Size: 500, Build Time: 15.5442 sec, Search Time: 0.4083 sec, Total Time: 15.9525 sec
Leaf Size: 600, Build Time: 15.5196 sec, Search Time: 0.4122 sec, Total Time: 15.9317 sec
Leaf Size: 700, Build Time: 15.8339 sec, Search Time: 0.4069 sec, Total Time: 16.2409 sec
Leaf Size: 800, Build Time: 15.4199 sec, Search Time: 0.4082 sec, Total Time: 15.8280 sec
Leaf Size: 900, Build Time: 15.8383 sec, Search Time: 0.4094 sec, Total Time: 16.24

**-----Opt Node Size-----**

In [ ]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array(
    [  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100, 1200,
  1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400,
  2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600,
  3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800,
  4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000,
  6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200,
  7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400,
  8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600,
  9700, 9800, 9900,10000]
)

# Total times from two runs (or versions)
total_times_1 = np.array(
    [18.1551,16.975 ,15.669 ,15.6471,14.8432,14.608 ,14.784 ,14.8412,14.8079,
 13.9931,14.0154,13.9359,13.9   ,13.8554,13.9923,13.9448,14.0226,14.141 ,
 13.9334,13.0201,13.0578,13.3141,13.0805,13.0716,13.1582,13.1068,13.2212,
 13.3349,13.4043,13.279 ,13.2633,13.3512,13.52  ,13.7125,13.3682,13.3559,
 13.2761,13.2766,13.238 ,12.4443,12.2948,12.4457,12.6325,12.4569,12.4691,
 12.3619,12.5393,12.5108,12.3329,12.6076,12.4673,12.4414,12.4781,12.6077,
 12.4272,12.4694,12.5804,12.5955,12.7565,12.6301,12.5758,12.6564,12.3953,
 12.6532,12.5891,12.3281,12.4796,12.4931,12.8581,12.8422,12.6458,12.8551,
 12.8688,12.65  ,12.7494,12.7736,12.7926,12.5012,11.6294,11.5535,11.6595,
 11.664 ,11.5817,11.6902,11.7175,11.6561,11.7222,11.5512,11.875 ,11.8607,
 11.8782,11.5954,11.6452,11.5196,11.7015,11.5705,11.7368,11.6474,11.7641,
 11.8892]
)

total_times_2 = np.array(
    [18.883 ,17.8042,17.0202,17.2489,15.9525,15.9317,16.2409,15.828 ,16.2477,
 15.2628,15.08  ,15.2124,15.0504,14.9958,14.8412,14.8986,14.9517,14.999 ,
 14.8973,14.189 ,14.1809,14.1699,14.1037,14.0927,14.0294,14.0928,14.1206,
 14.1716,14.1409,14.48  ,14.1154,14.18  ,13.8923,14.0828,14.0924,14.1485,
 14.0289,14.2279,14.3336,13.1554,13.0739,13.3688,13.157 ,13.2691,13.3327,
 13.0663,12.8844,12.8851,13.2172,13.2713,13.5315,13.1223,13.0826,13.047 ,
 13.0485,13.2281,13.3915,13.1069,13.0135,13.2299,13.1581,13.1628,12.9815,
 13.1919,13.2255,13.0957,13.2462,13.4149,13.5239,13.4125,13.2745,13.3457,
 13.2427,13.4344,13.1272,13.1982,13.4601,13.387 ,12.0792,12.0107,12.0154,
 11.7646,11.8504,11.906 ,11.898 ,11.9077,11.9675,11.9362,11.9849,11.9179,
 11.7068,11.9891,11.9412,11.9721,11.9673,11.8391,12.0612,11.9   ,11.8486,
 12.1539]
)

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 11.7048
Minimum found at leaf size: 9600


**-----------Epsilon Optimization For Rebuild---------**

**---------------Run 1---------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 9600
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 64.67it/s]


-------------------------------------------------------------
Build Time: 13.5927 sec, Search Time: 0.4733 sec
Epsilon: 0.0, Total Time: 14.0660 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 309.44it/s]


-------------------------------------------------------------
Build Time: 3.7728 sec, Search Time: 0.0855 sec
Epsilon: 0.5, Total Time: 3.8583 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 1.8210 sec, Search Time: 0.0031 sec
Epsilon: 29.0, Total Time: 1.8242 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 41262.21it/s]


-------------------------------------------------------------
Build Time: 1.8634 sec, Search Time: 0.0023 sec
Epsilon: 29.5, Total Time: 1.8658 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 26362.69it/s]

-------------------------------------------------------------
Build Time: 1.8862 sec, Search Time: 0.0026 sec
Epsilon: 30.0, Total Time: 1.8888 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 13.5927 sec, Search Time: 0.4733 sec, Total Time: 14.0660 sec
Epsilon: 0.5, Build Time: 3.7728 sec, Search Time: 0.0855 sec, Total Time: 3.8583 sec
Epsilon: 1.0, Build Time: 2.2684 sec, Search Time: 0.0279 sec, Total Time: 2.2963 sec
Epsilon: 1.5, Build Time: 1.9617 sec, Search Time: 0.0061 sec, Total Time: 1.9678 sec
Epsilon: 2.0, Build Time: 1.9572 sec, Search Time: 0.0183 sec, Total Time: 1.9754 sec
Epsilon: 2.5, Build Time: 1.8384 sec, Search Time: 0.0044 sec, Total Time: 1.8428 sec
Epsilon: 3.0, Build Time: 1.7880 sec, Search Time: 0.0038 sec, Total Time: 1.7918 sec
Epsilon: 3.5, Build Time: 1.8439 sec, Search Time: 0.0029 sec, Total Time: 1.8469 sec
Epsilon: 4.0, Build Time: 1.8763 sec, Search Time: 0.0047 sec, Total Time: 1.8809 sec
Epsilon: 4.5, Build Time: 1.92

**-------------Run 2------------------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 9600
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 71.38it/s]


-------------------------------------------------------------
Build Time: 12.8178 sec, Search Time: 0.4451 sec
Epsilon: 0.0, Total Time: 13.2629 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 252.94it/s]


-------------------------------------------------------------
Build Time: 3.8054 sec, Search Time: 0.0991 sec
Epsilon: 0.5, Total Time: 3.9045 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 1.8706 sec, Search Time: 0.0036 sec
Epsilon: 29.0, Total Time: 1.8743 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 54827.50it/s]


-------------------------------------------------------------
Build Time: 1.9204 sec, Search Time: 0.0032 sec
Epsilon: 29.5, Total Time: 1.9236 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 65741.44it/s]

-------------------------------------------------------------
Build Time: 1.8264 sec, Search Time: 0.0032 sec
Epsilon: 30.0, Total Time: 1.8296 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 12.8178 sec, Search Time: 0.4451 sec, Total Time: 13.2629 sec
Epsilon: 0.5, Build Time: 3.8054 sec, Search Time: 0.0991 sec, Total Time: 3.9045 sec
Epsilon: 1.0, Build Time: 2.3049 sec, Search Time: 0.0304 sec, Total Time: 2.3353 sec
Epsilon: 1.5, Build Time: 1.9777 sec, Search Time: 0.0078 sec, Total Time: 1.9855 sec
Epsilon: 2.0, Build Time: 1.8112 sec, Search Time: 0.0182 sec, Total Time: 1.8294 sec
Epsilon: 2.5, Build Time: 1.7274 sec, Search Time: 0.0061 sec, Total Time: 1.7335 sec
Epsilon: 3.0, Build Time: 1.7862 sec, Search Time: 0.0047 sec, Total Time: 1.7909 sec
Epsilon: 3.5, Build Time: 1.8287 sec, Search Time: 0.0028 sec, Total Time: 1.8315 sec
Epsilon: 4.0, Build Time: 1.8760 sec, Search Time: 0.0042 sec, Total Time: 1.8803 sec
Epsilon: 4.5, Build Time: 1.83

**-----------Epsilon Optimization For Rebuild------------**

In [ ]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ,25.5,26. ,26.5,27. ,27.5,
 28. ,28.5,29. ,29.5,30. ]
)

# Total times from two runs (or versions)
total_times_1 = np.array(
    [14.066 , 3.8583, 2.2963, 1.9678, 1.9754, 1.8428, 1.7918, 1.8469, 1.8809,
  1.9293, 1.8541, 1.909 , 1.8309, 1.8174, 1.884 , 1.8476, 1.8605, 1.8893,
  1.8298, 1.8902, 1.8742, 1.8672, 1.8489, 1.8533, 1.8397, 1.8049, 1.8537,
  1.7725, 1.8494, 1.9161, 1.8048, 1.8247, 1.7978, 1.8091, 1.8005, 1.8431,
  1.8874, 1.8218, 1.8189, 1.7922, 1.8125, 1.8285, 1.8709, 1.858 , 1.8467,
  1.8954, 1.9002, 1.8517, 1.9285, 1.7699, 1.8606, 1.8635, 1.7715, 1.825 ,
  1.8138, 1.8244, 1.8549, 1.8396, 1.8242, 1.8658, 1.8888]
)

total_times_2 = np.array(
    [13.2629, 3.9045, 2.3353, 1.9855, 1.8294, 1.7335, 1.7909, 1.8315, 1.8803,
  1.8399, 1.8697, 1.8717, 1.8861, 1.9677, 1.862 , 1.8936, 1.8653, 1.9191,
  1.9154, 1.9232, 1.8817, 1.8741, 1.8573, 1.9282, 1.879 , 1.9629, 1.8957,
  1.9186, 1.8719, 1.8947, 1.8912, 1.9173, 1.9289, 1.8869, 1.8382, 1.8539,
  1.8781, 1.9203, 1.9382, 1.8152, 1.9055, 1.8365, 1.8729, 1.8863, 1.9107,
  1.8736, 1.8727, 1.8643, 1.8851, 1.908 , 1.9217, 1.874 , 1.9347, 1.9042,
  1.8907, 1.8903, 1.9039, 1.8556, 1.8743, 1.9236, 1.8296]
)

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 1.7881
Minimum found at epsilon size: 2.5


**------Epsilon Optimization For Insertion------------**

**----------Run 1-----------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 24.11it/s]


-------------------------------------------------------------
Build Time: 8.7377 sec, Search Time: 1.0612 sec
Epsilon: 0.0, Total Time: 9.7990 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 32.87it/s]


-------------------------------------------------------------
Build Time: 8.4967 sec, Search Time: 1.0090 sec
Epsilon: 0.5, Total Time: 9.5056 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.5324 sec, Search Time: 1.1186 sec
Epsilon: 29.0, Total Time: 9.6510 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 24.07it/s]


-------------------------------------------------------------
Build Time: 8.6196 sec, Search Time: 1.1122 sec
Epsilon: 29.5, Total Time: 9.7318 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 24.35it/s]

-------------------------------------------------------------
Build Time: 8.5159 sec, Search Time: 1.1023 sec
Epsilon: 30.0, Total Time: 9.6182 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.7377 sec, Search Time: 1.0612 sec, Total Time: 9.7990 sec
Epsilon: 0.5, Build Time: 8.4967 sec, Search Time: 1.0090 sec, Total Time: 9.5056 sec
Epsilon: 1.0, Build Time: 8.6862 sec, Search Time: 1.1126 sec, Total Time: 9.7988 sec
Epsilon: 1.5, Build Time: 8.6679 sec, Search Time: 1.1442 sec, Total Time: 9.8121 sec
Epsilon: 2.0, Build Time: 8.8539 sec, Search Time: 1.1750 sec, Total Time: 10.0289 sec
Epsilon: 2.5, Build Time: 8.7619 sec, Search Time: 1.1299 sec, Total Time: 9.8918 sec
Epsilon: 3.0, Build Time: 8.6356 sec, Search Time: 1.1491 sec, Total Time: 9.7847 sec
Epsilon: 3.5, Build Time: 8.5441 sec, Search Time: 1.1758 sec, Total Time: 9.7199 sec
Epsilon: 4.0, Build Time: 8.7085 sec, Search Time: 1.1414 sec, Total Time: 9.8499 sec
Epsilon: 4.5, Build Time: 8.712

**----------Run 2---------------**

In [ ]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 9600
epsilon_rebuild = 2.5

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 30
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 23.89it/s]


-------------------------------------------------------------
Build Time: 8.7354 sec, Search Time: 1.1705 sec
Epsilon: 0.0, Total Time: 9.9060 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 27.74it/s]


-------------------------------------------------------------
Build Time: 8.7160 sec, Search Time: 1.1291 sec
Epsilon: 0.5, Total Time: 9.8452 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.6878 sec, Search Time: 1.1342 sec
Epsilon: 29.0, Total Time: 9.8220 sec

60: === Testing with Epsilon: 29.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 29.5): 100%|██████████| 20/20 [00:00<00:00, 23.84it/s]


-------------------------------------------------------------
Build Time: 9.0306 sec, Search Time: 1.1190 sec
Epsilon: 29.5, Total Time: 10.1496 sec

61: === Testing with Epsilon: 30.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 30.0): 100%|██████████| 20/20 [00:00<00:00, 23.54it/s]

-------------------------------------------------------------
Build Time: 8.8789 sec, Search Time: 1.1304 sec
Epsilon: 30.0, Total Time: 10.0093 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.7354 sec, Search Time: 1.1705 sec, Total Time: 9.9060 sec
Epsilon: 0.5, Build Time: 8.7160 sec, Search Time: 1.1291 sec, Total Time: 9.8452 sec
Epsilon: 1.0, Build Time: 8.6110 sec, Search Time: 1.2317 sec, Total Time: 9.8428 sec
Epsilon: 1.5, Build Time: 8.6639 sec, Search Time: 1.1978 sec, Total Time: 9.8617 sec
Epsilon: 2.0, Build Time: 8.6446 sec, Search Time: 1.1673 sec, Total Time: 9.8119 sec
Epsilon: 2.5, Build Time: 8.6491 sec, Search Time: 1.1652 sec, Total Time: 9.8143 sec
Epsilon: 3.0, Build Time: 8.8314 sec, Search Time: 1.1525 sec, Total Time: 9.9839 sec
Epsilon: 3.5, Build Time: 8.7526 sec, Search Time: 1.1700 sec, Total Time: 9.9226 sec
Epsilon: 4.0, Build Time: 8.6260 sec, Search Time: 1.1596 sec, Total Time: 9.7855 sec
Epsilon: 4.5, Build Time: 8.802

**--------Epsilon Optimization For Insert---------**

In [ ]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array(
    [ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ,25.5,26. ,26.5,27. ,27.5,
 28. ,28.5,29. ,29.5,30. ]
)

# Total times from two runs (or versions)
total_times_1 = np.array(
    [ 9.799 , 9.5056, 9.7988, 9.8121,10.0289, 9.8918, 9.7847, 9.7199, 9.8499,
  9.8551, 9.7498, 9.7465, 9.8653, 9.7643, 9.8055, 9.9772, 9.8155,10.1106,
  9.9089, 9.9667, 9.8582, 9.9171, 9.906 , 9.9306, 9.6327, 9.6896, 9.7519,
  9.8274, 9.8375, 9.8284, 9.7745, 9.7529, 9.8362, 9.8775, 9.6458, 9.837 ,
  9.9567, 9.7641, 9.8332, 9.8696, 9.7727, 9.6791, 9.8676, 9.9286, 9.7626,
  9.7745, 9.7693, 9.7527, 9.6951, 9.6361, 9.7421, 9.7686, 9.7924, 9.7879,
  9.8008, 9.8319, 9.6303, 9.718 , 9.651 , 9.7318, 9.6182]
)

total_times_2 = np.array(
    [ 9.906 , 9.8452, 9.8428, 9.8617, 9.8119, 9.8143, 9.9839, 9.9226, 9.7855,
  9.9802, 9.9228, 9.857 , 9.9108, 9.8571, 9.8711, 9.8581, 9.9285, 9.8118,
  9.953 , 9.9974, 9.7914, 9.9359, 9.8879, 9.9982, 9.8898, 9.9824, 9.8758,
  9.9222,10.169 , 9.8389, 9.873 ,10.012 , 9.9643, 9.8964, 9.9676, 9.9095,
  9.8844, 9.8547, 9.7568, 9.9181, 9.9178, 9.8692, 9.946 , 9.9327, 9.7365,
  9.8639, 9.7527, 9.8214, 9.697 , 9.886 , 9.9789, 9.8439, 9.7572, 9.7404,
  9.7437, 9.73  , 9.8276, 9.7534, 9.822 ,10.1496,10.0093]
)

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 9.6754
Minimum found at epsilon size: 0.5
